# Introduction

# Exact solution

We represent a weighted, non-oriented graph with an adjacency list. We also assume
that a graph is always connected, i.e. that it is always possible to reach a node from any
other node.

- *In a complete graph, all nodes are connected to one another. What is the number of Hamiltonian paths in a complete graph of size n?*

Among a set N of n nodes, we can describe each Hamilton path as an ordered combinaison of n ellements in N. It's an arrangment. In a complete graph, all nodes are conected so all arrangment n describe an Hamilton path.The number of possible arrangement ${\displaystyle A_{n}^{n}}=n!$ 

The number of Hamiltonian paths in a complete graph of size n is $n!$

# The Nearest Neighbor heuristic

# Conclusion